In [1]:
import pandas as pd
import numpy as np

df_train = pd.read_csv("/home/alex/dev/AiLearning/DataSetStore/text_classification_1/train.csv")
df_valid = pd.read_csv("/home/alex/dev/AiLearning/DataSetStore/text_classification_1/val.csv")

In [2]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181467 entries, 0 to 181466
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      181467 non-null  int64 
 1   text    181467 non-null  object
 2   class   181467 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.2+ MB


In [3]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [4]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22683 entries, 0 to 22682
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      22683 non-null  int64 
 1   text    22683 non-null  object
 2   class   22683 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 531.8+ KB


In [5]:
df_valid.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


# Message cleaner

In [6]:
import re
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk import tokenize as tknz
from nltk.corpus import stopwords

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

nltk.download('wordnet')

# Just copy from the homework of the lesson -1
class MessageCleaner:
    
    def __init__(self,message=None):
        self._message= message
        self.tokens=[]

    def set_message(self,msg):
        self._message = msg;
        return self

    def get_message(self):
        return self._message

    def remove_users(self,replace=" "):
        xtr = re.sub(pattern="@[\w]*",repl=replace,string=self._message)
        self._message = xtr
        return self
    
    def remove_punctuation(self,replace=" "):
        xtr = re.sub(pattern=r'[^\w\s]',repl=replace,string=self._message)
        self._message = xtr
        return self

    def remove_en_special_sym(self,replace=" "):
        xtr = re.sub(pattern=r'[^a-zA-Z0-9]',repl=replace,string=self._message)
        self._message = xtr
        return self

    def remove_ru_special_sym(self,replace=" "):
        xtr = re.sub(pattern=r'[^а-яА-Я0-9]',repl=replace,string=self._message)
        self._message = xtr
        return self
    
    def remove_en_numbers(self,replace=" "):
        xtr = re.sub(pattern=r'[^a-zA-Z]',repl=replace,string=self._message)
        self._message = xtr
        return self

    def remove_ru_numbers(self,replace=" "):
        xtr = re.sub(pattern=r'[^а-яА-Я]',repl=replace,string=self._message)
        self._message = xtr
        return self

    def clean_by_regexp(self,regexp: str):
        pattern = re.compile(regexp)
        xtr = pattern.subn(' ', self._message)[0]
        self._message = xtr
        return self

    def to_lower (self ):
        self._message = self._message.lower();
        return self;

    def replace_by_dicts (self,dictionary: map):
        for key in dictionary:
            self._message = self._message.replace(key,dictionary[key])
            #re.sub(pattern=key,repl=dictionary[key],string=self._message)
        return self

    def escape_single_symbol_words (self):
        self._message=" ".join([word for word in self._message.split() if len(word)>1])
        return self

    def nltk_word_tokenize(self):
        self.tokens=tknz.word_tokenize(self._message)
        return self

    def nltk_word_punc_tokenize(self):
        self.tokens = tknz.wordpunct_tokenize(self._message)
        return self

    def nltk_tok_tok_tokenizer(self):
        self.tokens = tknz.ToktokTokenizer().tokenize(self._message)
        return self
        
    def nltk_tweet_tokenizer(self):
        self.tokens = tknz.TweetTokenizer().tokenize(self._message)
        return self

    def nltk_with_regexp_tokenizer (self,regexp):
        self.tokens = tknz.RegexpTokenizer(regexp).tokenize(self._message)
        return self

    def nltk_sentence_tokenizer (self):
        self.tokens = nltk.sent_tokenize(self._message)
        return self

    def tokenize (self,tokenizer):
        self.tokens=tokenizer(self._message)
        return self

    def remove_stopwords_from_tokens (self,lang=None,is_new=True):
        if lang is None:
            sw = set(stopwords.words("english"))
        else :
            sw = set(stopwords.words(lang))
        tks = [token for token in self.tokens if token not in sw]
        if is_new:
            self.tokens_without_stops = tks
        else :
            self.tokens = tks
        return self

    def stemme_tokens (self,stemmer=None,is_new=True):
        if stemmer is None:
            stemmer = PorterStemmer()
        xtr = [stemmer.stem(token) for token in self.tokens]
        if is_new :
            self.stem_tokens = xtr
        else :
            self.tokens = xtr
        return self

    def lematize_tokens (self,lematizer=None,is_new=True):
        if lematizer is None:
            lematizer = WordNetLemmatizer()
        xtr = [lematizer.lemmatize(token) for token in self.tokens]
        if is_new:
            self.lem_tokens = xtr
        else :
            self.tokens = xtr
        return self

    def process_each_token(self,function):
        xtr = [function(token) for token in self.tokens]
        self.tokens = xtr
        return self

    def replace_message_by_sorted_tokens(self,token_array=None):
        if token_array is None:
            token_array = self.tokens
        xtr = " ".join(sorted(token_array))
        self._message = xtr
        return self

[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
from natasha import Segmenter,MorphVocab,Doc,NewsMorphTagger,NewsEmbedding

segmenter = Segmenter()
morph_tagger = NewsMorphTagger(NewsEmbedding())
morph_vocab = MorphVocab()

def make_natasha_tokens(text,
                        morphology_filter_set=None,
                        token_length_limit=None,
                        empty_token="empty",
                        verbose=False):
    
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    selected_tokens=[]
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
        if verbose:
            print (" >> Input {} ".format(token))
        new_token = None
        if morphology_filter_set is None:
            new_token = token.lemma
        else:
            if token.pos in morphology_filter_set:
                new_token = token.lemma
        if new_token is not None and token_length_limit is not None:
            if len(new_token)<token_length_limit:
                new_token = None
        if new_token is not None:
            selected_tokens.append(new_token)
        if verbose:
            print (" >> Output {} ".format(new_token))
    if len(selected_tokens)==0 :
        selected_tokens.append(empty_token)

    del doc
    return selected_tokens

# Message cleaner with Natasha and Nltk based approaches

In [8]:

def natasha_message_cleaner (text: str):
    return MessageCleaner()\
    .set_message(msg=text)\
    .to_lower()\
    .tokenize(tokenizer=lambda txt: make_natasha_tokens(text=txt,\
                                                        morphology_filter_set=('NOUN','VERB','ADJ','PUNCT','PRON'),\
                                                        token_length_limit=None))\
    .tokens

def nltk_message_cleaner(text: str):
    return MessageCleaner()\
    .set_message(msg=text)\
    .nltk_word_tokenize()\
    .lematize_tokens(is_new=False)\
    .remove_stopwords_from_tokens(lang="russian",is_new=False)\
    .tokens
    

# .nltk_word_tokenize()\
# .remove_punctuation()\   
# .remove_ru_numbers()\
# .remove_ru_special_sym()\
# .remove_ru_special_sym()\
# .lematize_tokens(is_new=False)\
# .remove_stopwords_from_tokens(lang="russian",is_new=False)\


# Just testing keras tokenizer with the Natasha-analyzer

In [9]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer


#num_words 	the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
#filters 	a string where each element is a character that will be filtered from the texts. The default is all punctuation, plus tabs and line breaks, minus the ' character.
#lower 	    boolean. Whether to convert the texts to lowercase.
#split 	    str. Separator for word splitting.
#char_level if True, every character will be treated as a token.
#oov_token 	if given, it will be added to word_index and used to replace out-of-vocabulary words during text_to_sequence calls
#analyzer 	function. Custom analyzer to split the text. The default analyzer is text_to_word_sequence 

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=None,
                      filters=None,
                      lower=None,
                      split=None,
                      char_level=False,
                      oov_token=None,
                      analyzer=natasha_message_cleaner)

print(tokenizer.to_json())

tokenizer.fit_on_texts(["Ну что сказать, я вижу","Кто-то наступил на грабли", "Ты разочаровал меня", "ты был натравлен"])
print("Document numbre = ",tokenizer.document_count)
print("Word counter: ",tokenizer.word_counts)
print("Word into docs counter: ",tokenizer.word_docs)
print("Index docs: ",tokenizer.index_docs)
print("Index word: ",tokenizer.index_word)
print("Word index: ",tokenizer.word_index)

print(tokenizer.texts_to_sequences(["Ну что сказать, я вижу кто-то наступил"]))


2024-01-04 04:57:34.248376: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-04 04:57:34.249634: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-04 04:57:34.269592: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 04:57:34.269612: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 04:57:34.269627: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

{"class_name": "Tokenizer", "config": {"num_words": null, "filters": null, "lower": null, "split": null, "char_level": false, "oov_token": null, "document_count": 0, "word_counts": "{}", "word_docs": "{}", "index_docs": "{}", "index_word": "{}", "word_index": "{}"}}
Document numbre =  4
Word counter:  OrderedDict([('что', 1), ('сказать', 1), (',', 1), ('я', 2), ('видеть', 1), ('кто-то', 1), ('наступить', 1), ('грабли', 1), ('ты', 2), ('разочаровать', 1), ('натравить', 1)])
Word into docs counter:  defaultdict(<class 'int'>, {'сказать': 1, 'я': 2, 'что': 1, 'видеть': 1, ',': 1, 'наступить': 1, 'грабли': 1, 'кто-то': 1, 'разочаровать': 1, 'ты': 2, 'натравить': 1})
Index docs:  defaultdict(<class 'int'>, {4: 1, 1: 2, 3: 1, 6: 1, 5: 1, 8: 1, 9: 1, 7: 1, 10: 1, 2: 2, 11: 1})
Index word:  {1: 'я', 2: 'ты', 3: 'что', 4: 'сказать', 5: ',', 6: 'видеть', 7: 'кто-то', 8: 'наступить', 9: 'грабли', 10: 'разочаровать', 11: 'натравить'}
Word index:  {'я': 1, 'ты': 2, 'что': 3, 'сказать': 4, ',': 5, '

# Calculating X,Y with the Natasha tokenizer

In [10]:
train_messages = df_train['text'].values
valid_messages = df_valid['text'].values

ANALYZER = natasha_message_cleaner

tokenizer = Tokenizer(num_words=None,
                      filters=None,
                      lower=None,
                      split=None,
                      char_level=False,
                      oov_token=None,
                      analyzer=ANALYZER)

tokenizer.fit_on_texts(train_messages)

train_vectors = tokenizer.texts_to_sequences(train_messages)
valid_vectors = tokenizer.texts_to_sequences(valid_messages)

max_vector_length = max([len(ANALYZER(train_message)) for train_message in train_messages])
dictionary_size_plus_1 = len(tokenizer.index_word) + 1


# https://www.tensorflow.org/api_docs/python/tf/keras/utils/pad_sequences
# just adding zeroes to vectors with small sizes
from keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(train_vectors, maxlen=max_vector_length)
x_valid = pad_sequences(valid_vectors, maxlen=max_vector_length)
y_train = df_train['class'].values
y_valid = df_valid['class'].values

print (max_vector_length,dictionary_size_plus_1)
print (x_train.shape)


130 93277
(181467, 130)


# Just testing keras embedding

In [11]:
# https://keras.io/api/layers/core_layers/embedding/
from keras.layers import Embedding
from keras.models import Sequential

#    input_dim: Integer. Size of the vocabulary, i.e. maximum integer index + 1.
#    output_dim: Integer. Dimension of the dense embedding.
#    embeddings_initializer: Initializer for the embeddings matrix (see keras.initializers).
#    embeddings_regularizer: Regularizer function applied to the embeddings matrix (see keras.regularizers).
#    embeddings_constraint: Constraint function applied to the embeddings matrix (see keras.constraints).
#    mask_zero: Boolean, whether or not the input value 0 is a special "padding" value that should be masked out. This is useful when using recurrent layers which may take variable length input. If this is True, then all subsequent layers in the model need to support masking or an exception will be raised. If mask_zero is set to True, as a consequence, index 0 cannot be used in the vocabulary (input_dim should equal size of vocabulary + 1).

embedding = Embedding(
    input_dim=dictionary_size_plus_1,
    output_dim=max_vector_length,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    embeddings_constraint=None,
    mask_zero=True
)
model = Sequential()
model.add(embedding)

print (model.predict(x_train[0]).shape)
print (x_train[0].shape)



5/5 [==============================] - 0s 622us/step
(130, 130)
(130,)


2024-01-04 05:18:22.478710: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-04 05:18:22.491210: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# RNN-based approach

In [12]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import SimpleRNN,Dropout,Dense,BatchNormalization
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError

#https://keras.io/api/layers/recurrent_layers/simple_rnn/

rnn = Sequential()
rnn.add(Embedding(
    input_dim=dictionary_size_plus_1,
    output_dim=max_vector_length,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    embeddings_constraint=None,
    mask_zero=True))

rnn.add(SimpleRNN(units=max_vector_length,dropout=0.25,recurrent_dropout=0.4))
rnn.add(Dense(96, activation='relu'))
rnn.add(BatchNormalization())
rnn.add(Dropout(0.75))
rnn.add(Dense(96, activation='tanh'))
rnn.add(BatchNormalization())
rnn.add(Dropout(0.5))
rnn.add(Dense(96, activation='relu'))
rnn.add(BatchNormalization())
rnn.add(Dropout(0.25))
rnn.add(Dense(96, activation='tanh'))
rnn.add(BatchNormalization())
rnn.add(Dropout(0.15))
rnn.add(Dense(1, activation='sigmoid'))

rnn.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['accuracy'])
rnn.summary()

# https://keras.io/api/callbacks/early_stopping/
early_stopping=EarlyStopping(monitor='val_accuracy',
                             baseline=0.70,
                             patience=20,
                             restore_best_weights=True,
                             start_from_epoch=3,
                             mode='max',
                             verbose=1)  

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 130)         12126010  
                                                                 
 simple_rnn (SimpleRNN)      (None, 130)               33930     
                                                                 
 dense (Dense)               (None, 96)                12576     
                                                                 
 batch_normalization (Batch  (None, 96)                384       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 96)                0         
                                                                 
 dense_1 (Dense)             (None, 96)                9312      
                                                      

In [13]:
history = rnn.fit(x_train, y_train,
                    batch_size=2000,
                    epochs=50,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/50
73/73 [==============================] - 36s 469ms/step - loss: 0.2600 - accuracy: 0.5674 - val_loss: 0.0906 - val_accuracy: 0.8941
Epoch 2/50
73/73 [==============================] - 34s 468ms/step - loss: 0.0713 - accuracy: 0.9043 - val_loss: 0.0461 - val_accuracy: 0.9515
Epoch 3/50
73/73 [==============================] - 34s 467ms/step - loss: 0.0256 - accuracy: 0.9684 - val_loss: 0.0420 - val_accuracy: 0.9561
Epoch 4/50
73/73 [==============================] - 34s 468ms/step - loss: 0.0177 - accuracy: 0.9788 - val_loss: 0.0245 - val_accuracy: 0.9743
Epoch 5/50
73/73 [==============================] - 34s 468ms/step - loss: 0.0139 - accuracy: 0.9836 - val_loss: 0.0222 - val_accuracy: 0.9764
Epoch 6/50
73/73 [==============================] - 34s 469ms/step - loss: 0.0119 - accuracy: 0.9861 - val_loss: 0.0182 - val_accuracy: 0.9808
Epoch 7/50
73/73 [==============================] - 34s 468ms/step - loss: 0.0101 - accuracy: 0.9883 - val_loss: 0.0181 - val_accuracy: 0.9806

In [14]:
score = rnn.evaluate(x_valid, y_valid, batch_size=512, verbose=1)
print (score)
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 35ms/step - loss: 0.0128 - accuracy: 0.9862
[0.012773622758686543, 0.9862452149391174]
Test score: 0.012773622758686543
Test accuracy: 0.9862452149391174


# LSTM -based approach

In [15]:
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense

# https://keras.io/api/layers/recurrent_layers/lstm/

lstm = Sequential()
lstm.add(Embedding(
    input_dim=dictionary_size_plus_1,
    output_dim=max_vector_length,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    embeddings_constraint=None,
    mask_zero=True))

lstm.add(LSTM(units=max_vector_length,dropout=0.3,activation="tanh",
    recurrent_activation="tanh",recurrent_dropout=0.3))
lstm.add(Dense(96, activation='relu'))
lstm.add(BatchNormalization())
lstm.add(Dropout(0.65))
lstm.add(Dense(96, activation='tanh'))
lstm.add(BatchNormalization())
lstm.add(Dropout(0.45))
lstm.add(Dense(96, activation='relu'))
lstm.add(BatchNormalization())
lstm.add(Dropout(0.20))
lstm.add(Dense(96, activation='tanh'))
lstm.add(BatchNormalization())
lstm.add(Dropout(0.15))
lstm.add(Dense(1, activation='sigmoid'))

lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm.summary()

# https://keras.io/api/callbacks/early_stopping/
early_stopping=EarlyStopping(monitor='val_accuracy',
                             baseline=0.70,
                             patience=20,
                             restore_best_weights=True,
                             start_from_epoch=3,
                             mode='max',
                             verbose=1)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 130)         12126010  
                                                                 
 lstm (LSTM)                 (None, 130)               135720    
                                                                 
 dense_5 (Dense)             (None, 96)                12576     
                                                                 
 batch_normalization_4 (Bat  (None, 96)                384       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 96)                0         
                                                                 
 dense_6 (Dense)             (None, 96)                9312      
                                                      

In [16]:
history = lstm.fit(x_train, y_train,
                    batch_size=2000,
                    epochs=50,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/50
73/73 [==============================] - 97s 1s/step - loss: 0.2064 - accuracy: 0.8994 - val_loss: 0.7218 - val_accuracy: 0.5070
Epoch 2/50
73/73 [==============================] - 93s 1s/step - loss: 0.0379 - accuracy: 0.9868 - val_loss: 0.9230 - val_accuracy: 0.5070
Epoch 3/50
73/73 [==============================] - 93s 1s/step - loss: 0.0208 - accuracy: 0.9934 - val_loss: 1.1741 - val_accuracy: 0.5071
Epoch 4/50
73/73 [==============================] - 94s 1s/step - loss: 0.0155 - accuracy: 0.9952 - val_loss: 0.9820 - val_accuracy: 0.5129
Epoch 5/50
73/73 [==============================] - 93s 1s/step - loss: 0.0116 - accuracy: 0.9966 - val_loss: 0.7139 - val_accuracy: 0.5656
Epoch 6/50
73/73 [==============================] - 93s 1s/step - loss: 0.0087 - accuracy: 0.9973 - val_loss: 0.3206 - val_accuracy: 0.8279
Epoch 7/50
73/73 [==============================] - 94s 1s/step - loss: 0.0080 - accuracy: 0.9975 - val_loss: 0.0759 - val_accuracy: 0.9770
Epoch 8/50
73/73 [==

In [17]:
score = lstm.evaluate(x_valid, y_valid, batch_size=512, verbose=1)
print (score)
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 3s 76ms/step - loss: 0.0821 - accuracy: 0.9862
[0.08211026340723038, 0.9862452149391174]
Test score: 0.08211026340723038
Test accuracy: 0.9862452149391174


# Just testing convolutional layer

In [18]:
from keras.layers import Conv1D,Flatten

#filters:       int, the dimension of the output space (the number of filters in the convolution).
#kernel_size:   int or tuple/list of 1 integer, specifying the size of the convolution window.
#strides:       int or tuple/list of 1 integer, specifying the stride length of the convolution.
#               strides > 1 is incompatible with dilation_rate > 1.
#padding:       string, "valid", "same" or "causal"(case-insensitive). "valid" means no padding.
#               "same" results in padding evenly to the left/right or up/down of the input.
#               When padding="same" and strides=1, the output has the same size as the input.
#               "causal" results in causal(dilated) convolutions, e.g. output[t] does not depend oninput[t+1:].
#               Useful when modeling temporal data where the model should not violate the temporal order.
#               See WaveNet: A Generative Model for Raw Audio, section2.1.
#data_format:   string, either "channels_last" or "channels_first". The ordering of the dimensions in the inputs.
#               "channels_last" corresponds to inputs with shape (batch, steps, features) while "channels_first"
#               corresponds to inputs with shape (batch, features, steps). It defaults to the image_data_format
#               value found in your Keras config file at ~/.keras/keras.json. If you never set it, then it will be "channels_last".
#dilation_rate: int or tuple/list of 1 integers, specifying the dilation rate to use for dilated convolution.
#groups:        A positive int specifying the number of groups in which the input is split along the channel axis.
#               Each group is convolved separately with filters // groups filters. The output is the concatenation
#               of all the groups results along the channel axis. Input channels and filters must both be divisible by groups.
#activation:    Activation function. If None, no activation is applied.


conv_layer = Conv1D(
    filters=1,
    kernel_size=3,
    strides=1,
    padding="same",
    data_format="channels_first",
    dilation_rate=1,
    groups=1,
    activation=None,
    use_bias=False
)

array  = np.array([[[1,0,0,0,0],
                    [0,1,0,0,0],
                    [0,0,1,0,0],
                    [0,0,0,1,0],
                    [0,0,0,0,1]]])

# array  = np.array([[[0,0,0,0,0],
#                    [-11,-12,-13,-14,-15],
#                    [0,0,0,0,0],
#                    [-11,-12,-13,-14,-15],
#                    [0,0,0,0,0]]])

conv = Sequential()
conv.add(conv_layer)
conv.add(Flatten(data_format="channels_first"))
print (conv.predict(array.astype(float)))
print (conv_layer.get_weights())


1/1 [==============================] - 0s 40ms/step
[[ 0.7524413   0.2218785   0.0165498  -0.37880778  0.37230915]]
[array([[[-2.07834870e-01],
        [ 3.66568565e-04],
        [-3.01807165e-01],
        [ 4.93916273e-01],
        [ 4.03601766e-01]],

       [[ 4.05539751e-01],
        [ 4.73249197e-01],
        [-2.38217086e-01],
        [ 2.89316595e-01],
        [-1.21607125e-01]],

       [[ 3.37714434e-01],
        [ 3.46901536e-01],
        [-4.35358286e-02],
        [ 2.54400313e-01],
        [-3.66317213e-01]]], dtype=float32)]


# Conv1d NN

In [19]:
from keras.layers import Reshape
#import keras

# Convolutional 1D from 130x to 4x
conv = Sequential()
conv.add(Embedding(
    input_dim=dictionary_size_plus_1,
    output_dim=max_vector_length,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    embeddings_constraint=None,
    mask_zero=True))
conv.add(Reshape(target_shape=(130,130)))
# Decresing from 130 to 128
conv.add(Conv1D(filters=8, kernel_size=3,strides=1,padding='valid',activation='relu'))
conv.add(Conv1D(filters=8, kernel_size=3,strides=1,padding='same',activation='relu'))
conv.add(BatchNormalization())
conv.add(Dropout(0.25))
conv.add(Conv1D(filters=8, kernel_size=2,strides=2,padding='same',activation='relu'))
conv.add(Conv1D(filters=8, kernel_size=3,strides=1,padding='same',activation='relu'))
conv.add(BatchNormalization())
conv.add(Dropout(0.25))
conv.add(Conv1D(filters=8, kernel_size=2,strides=2,padding='same',activation='relu'))
conv.add(Conv1D(filters=8, kernel_size=3,strides=1,padding='same',activation='relu'))
conv.add(BatchNormalization())
conv.add(Dropout(0.25))
conv.add(Conv1D(filters=16, kernel_size=2,strides=2,padding='same',activation='relu'))
conv.add(Conv1D(filters=16, kernel_size=3,strides=1,padding='same',activation='relu'))
conv.add(BatchNormalization())
conv.add(Dropout(0.25))
conv.add(Conv1D(filters=16, kernel_size=2,strides=2,padding='same',activation='relu'))
conv.add(Conv1D(filters=16, kernel_size=3,strides=1,padding='same',activation='relu'))
conv.add(BatchNormalization())
conv.add(Dropout(0.25))
conv.add(Conv1D(filters=16, kernel_size=2,strides=2,padding='same',activation='relu'))
conv.add(Conv1D(filters=16, kernel_size=3,strides=1,padding='same',activation='relu'))
conv.add(Reshape(target_shape=(4*16,)))
conv.add(Dense(96, activation='relu'))
conv.add(BatchNormalization())
conv.add(Dropout(0.65))
conv.add(Dense(96, activation='tanh'))
conv.add(BatchNormalization())
conv.add(Dropout(0.45))
conv.add(Dense(96, activation='relu'))
conv.add(BatchNormalization())
conv.add(Dropout(0.20))
conv.add(Dense(96, activation='tanh'))
conv.add(BatchNormalization())
conv.add(Dropout(0.15))
conv.add(Dense(1, activation='sigmoid'))

conv.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['accuracy'])
conv.summary()

# https://keras.io/api/callbacks/early_stopping/
early_stopping=EarlyStopping(monitor='val_accuracy',
                             baseline=0.70,
                             patience=20,
                             restore_best_weights=True,
                             start_from_epoch=50,
                             mode='max',
                             verbose=1)  

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 130)         12126010  
                                                                 
 reshape (Reshape)           (None, 130, 130)          0         
                                                                 
 conv1d_1 (Conv1D)           (None, 128, 8)            3128      
                                                                 
 conv1d_2 (Conv1D)           (None, 128, 8)            200       
                                                                 
 batch_normalization_8 (Bat  (None, 128, 8)            32        
 chNormalization)                                                
                                                                 
 dropout_8 (Dropout)         (None, 128, 8)            0         
                                                      

In [20]:
history = conv.fit(x_train, y_train,
                    batch_size=500,
                    epochs=100,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/100
291/291 [==============================] - 24s 73ms/step - loss: 0.2507 - accuracy: 0.5652 - val_loss: 0.4513 - val_accuracy: 0.4930
Epoch 2/100
291/291 [==============================] - 21s 72ms/step - loss: 0.0608 - accuracy: 0.9233 - val_loss: 0.0362 - val_accuracy: 0.9579
Epoch 3/100
291/291 [==============================] - 21s 72ms/step - loss: 0.0243 - accuracy: 0.9717 - val_loss: 0.0154 - val_accuracy: 0.9831
Epoch 4/100
291/291 [==============================] - 21s 72ms/step - loss: 0.0187 - accuracy: 0.9787 - val_loss: 0.0145 - val_accuracy: 0.9836
Epoch 5/100
291/291 [==============================] - 21s 72ms/step - loss: 0.0150 - accuracy: 0.9831 - val_loss: 0.0137 - val_accuracy: 0.9850
Epoch 6/100
291/291 [==============================] - 21s 72ms/step - loss: 0.0128 - accuracy: 0.9855 - val_loss: 0.0123 - val_accuracy: 0.9862
Epoch 7/100
291/291 [==============================] - 21s 72ms/step - loss: 0.0110 - accuracy: 0.9878 - val_loss: 0.0144 - val_ac

In [21]:
score = conv.evaluate(x_valid, y_valid, batch_size=512, verbose=1)
print (score)
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 0s 7ms/step - loss: 0.0050 - accuracy: 0.9947
[0.0050274827517569065, 0.9947096705436707]
Test score: 0.0050274827517569065
Test accuracy: 0.9947096705436707


# Conv2d NN

In [22]:
from keras.layers import Conv2D,Flatten,Reshape

# Convolutional 1D from 130x to 4x
conv2 = Sequential()
conv2.add(Embedding(
    input_dim=dictionary_size_plus_1,
    output_dim=max_vector_length,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    embeddings_constraint=None,
    mask_zero=True))
conv2.add(Reshape(target_shape=(130,130,1)))
# Decresing from 130 to 128
conv2.add(Conv2D(filters=8, kernel_size=(3,3),strides=1,padding='valid',activation='relu'))
conv2.add(Conv2D(filters=8, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.25))
conv2.add(Conv2D(filters=8, kernel_size=(2,2),strides=2,padding='same',activation='relu'))
conv2.add(Conv2D(filters=8, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.25))
conv2.add(Conv2D(filters=8, kernel_size=(2,2),strides=2,padding='same',activation='relu'))
conv2.add(Conv2D(filters=8, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.25))
conv2.add(Conv2D(filters=16, kernel_size=(2,2),strides=2,padding='same',activation='relu'))
conv2.add(Conv2D(filters=16, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.25))
conv2.add(Conv2D(filters=16, kernel_size=(2,2),strides=2,padding='same',activation='relu'))
conv2.add(Conv2D(filters=16, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.25))
conv2.add(Conv2D(filters=32, kernel_size=(2,2),strides=2,padding='same',activation='relu'))
conv2.add(Conv2D(filters=32, kernel_size=(3,3),strides=1,padding='same',activation='relu'))
conv2.add(Flatten())
conv2.add(Dense(96, activation='relu'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.65))
conv2.add(Dense(96, activation='tanh'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.45))
conv2.add(Dense(96, activation='relu'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.20))
conv2.add(Dense(96, activation='tanh'))
conv2.add(BatchNormalization())
conv2.add(Dropout(0.15))
conv2.add(Dense(1, activation='sigmoid'))

conv2.compile(optimizer='adam', loss=MeanSquaredError(), metrics=['accuracy'])
conv2.summary()

# https://keras.io/api/callbacks/early_stopping/
early_stopping=EarlyStopping(monitor='val_accuracy',
                             baseline=0.70,
                             patience=10,
                             restore_best_weights=True,
                             start_from_epoch=20,
                             mode='max',
                             verbose=1)  

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 130)         12126010  
                                                                 
 reshape_2 (Reshape)         (None, 130, 130, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 8)       80        
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 8)       584       
                                                                 
 batch_normalization_17 (Ba  (None, 128, 128, 8)       32        
 tchNormalization)                                               
                                                                 
 dropout_17 (Dropout)        (None, 128, 128, 8)       0         
                                                      

In [23]:
history = conv2.fit(x_train, y_train,
                    batch_size=500,
                    epochs=50,
                    verbose=1,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/50
291/291 [==============================] - 171s 581ms/step - loss: 0.2684 - accuracy: 0.5005 - val_loss: 0.2505 - val_accuracy: 0.5070
Epoch 2/50
291/291 [==============================] - 169s 580ms/step - loss: 0.2547 - accuracy: 0.5004 - val_loss: 0.2510 - val_accuracy: 0.5070
Epoch 3/50
291/291 [==============================] - 169s 580ms/step - loss: 0.2511 - accuracy: 0.5024 - val_loss: 0.2503 - val_accuracy: 0.5070
Epoch 4/50
291/291 [==============================] - 169s 580ms/step - loss: 0.2499 - accuracy: 0.5132 - val_loss: 0.2407 - val_accuracy: 0.5865
Epoch 5/50
291/291 [==============================] - 169s 580ms/step - loss: 0.1107 - accuracy: 0.8184 - val_loss: 0.0092 - val_accuracy: 0.9901
Epoch 6/50
291/291 [==============================] - 169s 580ms/step - loss: 0.0081 - accuracy: 0.9906 - val_loss: 0.0061 - val_accuracy: 0.9932
Epoch 7/50
291/291 [==============================] - 169s 579ms/step - loss: 0.0049 - accuracy: 0.9945 - val_loss: 0.0053 -

In [24]:
score = conv2.evaluate(x_valid, y_valid, batch_size=512, verbose=1)
print (score)
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 4s 90ms/step - loss: 0.0045 - accuracy: 0.9952
[0.004482968710362911, 0.995238721370697]
Test score: 0.004482968710362911
Test accuracy: 0.995238721370697
